
### **Bias-correcting and downscaling  temperature, precipitation, and wind speed**

##### This jupyter notebook runs through the steps for bias correcting and downscaling output from the EC-Earth3-Veg Global Climate Model under CMIP6.

Author: Adam Young  
Date: 2023-02-10  
Contact: adam.m.young@outlook.com  

--------------------------------------

The steps for this example are as follows:  
1. Load in required libraries
2. Read in 30 yr of data for our:  
    - Reference Data Set (ERA5, 1980-2009)  
    - Historical GCM output (EC-Earth3-Veg, 1980-2009)  
    - Future projected GCM output (EC-Earth3-Veg, 2030-2059)  

    The above datasets are all read in NetCDF format using the [xarray][1] library  
3. For select grid cells/locations, apply a quantile delta mapping approach ([Cannon et al. 2015][2]) to bias correct near-surface air temperature, daily precipitation, and 10-metre wind speed. Use basic stats and numpy packages at first to walk through the individual steps. 
4. Compare results from step 3 to bias corrections derived using the [xclim][3] library.  

[1]: <https://docs.xarray.dev>
[2]: <https://doi.org/10.1175/JCLI-D-14-00754.1>
[3]: <https://github.com/Ouranosinc/xclim>


First, import required packages ...

In [63]:
from pathlib import Path

import numpy as np

# Module for bias correcting from xclim
from xclim.sdba import QuantileDeltaMapping

# Needed to account for low precip values
from xclim.sdba.processing import jitter_under_thresh 
import xarray as xr

Next, read in the following three datasets ...  

 - ref: reference observational/reanalysis dataset, here [ERA5][4]. Use years of 1980-2009 for reference time period
 - hst: historical GCM data for [EC-Earth3-Veg][5] for the same time period as ERA5 (1080-2009)
 - prj: projected GCM output for EC-Earth3-Veg for 2030-2059 and 2060-2089

As a brief aside, EC-Earth3-Veg was determined to have the highest ranking among 101 GCM-ensemble combinations for Alaska using the [gcmeval tool][6].

[4]: <https://www.ecmwf.int/en/forecasts/datasets/reanalysis-datasets/era5>
[5]: <https://doi.org/10.5194/gmd-15-2973-2022>
[6]: <https://gcmeval.met.no>

In [64]:
# Get current working directory based on location of current ipynb file in the
# notebooks folder and then link to path for already pre-processed ERA5 and GCM
# datasets.
currpath = Path.cwd()
datapath = currpath.joinpath('../data')
era5_datapath = datapath.joinpath('processed/climate/era5')
gcm_datapath = datapath.joinpath('processed/climate/cmip6/EC-Earth3-Veg')

# Set yr ranges
hst_yr = np.arange(1980,2010) # Historical years for reference
prj_yr = np.arange(2030,2060) # Projected years

# The following lines of code are commented out, but they would provide the 
# method for loading the spatial datasets and extracting all data needed to 
# conduct the analysis for a single location/grid cell.

ref_filelist = list(era5_datapath.glob('tas_*')) \
               + list(era5_datapath.glob('pr_*')) \
               + list(era5_datapath.glob('sfcWind_*'))
gcm_filelist = list(gcm_datapath.glob('tas_*')) \
               + list(gcm_datapath.glob('pr_*')) \
               + list(gcm_datapath.glob('sfcWind_*'))

# Use parallel=True below to initiate use of dask arrays and parallel 
# computing. This is likely necessary on personal computers due to memory 
# needs. Separate out gcm into both historical and projected variables based on
# years/time periods. Then delete gcm variable ('del gcm') as no longer needed.
#
ref = xr.open_mfdataset(ref_filelist,engine='h5netcdf',parallel=True)
gcm = xr.open_mfdataset(gcm_filelist,engine='h5netcdf',parallel=True)

hst = gcm.sel(time=slice(str(hst_yr[0]),str(hst_yr[-1])))
prj = gcm.sel(time=slice(str(prj_yr[0]),str(prj_yr[-1])))

del gcm

TypeError: '>' not supported between instances of 'int' and 'cftime._cftime.DatetimeNoLeap'

Next, we will linearly interpolate the GCM data to the resolution of ERA5. They need to be on the same grid system and align with each other to do the bias corrections. This is also commented out for the sake of 

In [ ]:
# >>> ref_lat = ref['lat'].values
# >>> ref_lon = ref['lon'].values
#
# >>> hst = hst.interp(lat=ref_lat,lon=ref_lon,method="linear")
# >>> prj = prj.interp(lat=ref_lat,lon=ref_lon,method="linear")

Next, filter the datasets to only a single grid cell based on latitude and longitude. Here we will use Fairbanks, Alaska.

In [ ]:
coords = dict(lon=-147.72,lat=64.8401) # Coordinates for Fairbanks AK
# >>> ref_pt = ref.sel(lat=coords['lat'],lon=coords['lon'],method="nearest")
# >>> hst_pt = hst.sel(lat=coords['lat'],lon=coords['lon'],method="nearest")
# >>> prj_pt = prj.sel(lat=coords['lat'],lon=coords['lon'],method="nearest")

NameError: name 'x' is not defined

#### References
- xarray:               <https://docs.xarray.dev>  
- Cannon et al. 2015:   <https://doi.org/10.1175/JCLI-D-14-00754.1>  
- xclim:                <https://github.com/Ouranosinc/xclim>  
- ERA5:                 <https://www.ecmwf.int/en/forecasts/datasets/reanalysis-datasets/era5>  
- EC-Earth3-Veg:        <https://doi.org/10.5194/gmd-15-2973-2022>
- GCM-eval tool:        <https://gcmeval.met.no>